# Web for CREST flood quantile

## 1. Environment

In [10]:
! pip install pandas geopandas folium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.3/102.3 kB 1.4 MB/s eta 0:00:00a 0:00:01


# 2. Read data

### 2.1 Read streamflow forecast data as pandas dataframe.

In [29]:
import os
import glob
import pandas as pd
import geopandas as gpd

# Initialize an empty dataframe to store the peak values, corresponding dates, and raw data paths
df = pd.DataFrame(columns=['Peak_R', 'Date', 'OutName', 'RawDataPath'])

# Define the directory path
dir_path = './demo_project/Web_data/2023072600-2023073112'

# Use os.walk to traverse the directory tree
for root, dirs, files in os.walk(dir_path):
    for file in files:
        # Check if the file is a .csv file
        if file.endswith('.csv'):
            # Construct the full file path
            file_path = os.path.join(root, file)

            temp_df = pd.read_csv(file_path)

            # Drop rows with NaN in 'R' column
            temp_df = temp_df.dropna(subset=['R'])

            # Get the peak value of the R column and its corresponding date
            peak_r = temp_df['R'].max()
            peak_date = temp_df.loc[temp_df['R'].idxmax(), 'Date']

            # Extract the file name without the extension
            outname = os.path.splitext(file)[0]

            # Create a new dataframe with the new data
            new_df = pd.DataFrame({'Peak_R': [peak_r], 'Date': [peak_date], 'OutName': [outname], 'RawDataPath': [file_path]})

            # Append the new data to the main dataframe
            df = pd.concat([df, new_df], ignore_index=True)

# Print the number of records before removal
print("Number of records before removal: ", len(df))

# Remove rows where Peak_R == 0
df = df[df['Peak_R'] != 0].reset_index(drop=True)

# Print the number of records after removal
print("Number of records after removal: ", len(df))

# quick look at the data
pd.set_option('display.max_colwidth', None)
print(df)


Number of records before removal:  750
Number of records after removal:  114
       Peak_R           Date    OutName  \
0    1.726819  2023/07/30:13   378102_5   
1     0.82098  2023/07/31:05  378102_32   
2    0.004452  2023/07/30:19     SWC_37   
3    0.004181  2023/07/30:20     SWC_36   
4    0.798008  2023/07/31:07  378102_33   
..        ...            ...        ...   
109  1.183886  2023/07/30:15  380219_24   
110  1.158825  2023/07/30:19  378102_16   
111  1.176541  2023/07/30:19  378102_17   
112  2.473001  2023/07/30:21   379520_7   
113  0.942899  2023/07/30:18  380219_19   

                                                                                                                   RawDataPath  
0     /Users/qyang/Downloads/data/CREST/CREST_tutorial/demo_project/Web_data/2023072600-2023073112/SouthWestCoast/378102_5.csv  
1    /Users/qyang/Downloads/data/CREST/CREST_tutorial/demo_project/Web_data/2023072600-2023073112/SouthWestCoast/378102_32.csv  
2       /Users/qyan

### 3.2 Read the outlet shapefiles as geopands dataframe.

In [30]:
import geopandas as gpd
# Load the shapefile using Geopandas
gdf = gpd.read_file('./demo_project/Web_data/SouthWestCoast/SouthWestCoast_Out.shp')

# display the geodataframe
print('number of outlets: ', len(gdf))
print(gdf)


number of outlets:  266
      OutName                   geometry
0    377441_1  POINT (41.27737 18.61450)
1    377441_2  POINT (41.40699 18.74782)
2    377441_3  POINT (41.74910 18.72719)
3    377441_4  POINT (41.81145 18.75575)
4    377441_5  POINT (42.06917 18.55240)
..        ...                        ...
261  379520_4  POINT (42.56930 17.42668)
262  379520_5  POINT (42.56930 17.47724)
263  379520_6  POINT (42.64812 17.47635)
264  379520_7  POINT (42.70368 17.76474)
265  379520_8  POINT (42.57290 17.34824)

[266 rows x 2 columns]


### 3.3 Merge the data and locations.

In [31]:
# Merge the main dataframe with the geodataframe based on 'OutName'
merged = gdf.merge(df, on='OutName')
print(merged.head())

     OutName                   geometry    Peak_R           Date  \
0   370483_6  POINT (42.52772 18.96002)  3.754707  2023/07/30:08   
1   370483_8  POINT (42.62765 18.80982)  4.136328  2023/07/30:05   
2   370483_9  POINT (42.66126 18.75211)  4.266354  2023/07/30:04   
3  370483_10  POINT (42.72759 18.55640)  4.946623  2023/07/30:00   
4  370483_13  POINT (42.69844 18.45617)   5.01051  2023/07/29:22   

                                                                                                                 RawDataPath  
0   /Users/qyang/Downloads/data/CREST/CREST_tutorial/demo_project/Web_data/2023072600-2023073112/SouthWestCoast/370483_6.csv  
1   /Users/qyang/Downloads/data/CREST/CREST_tutorial/demo_project/Web_data/2023072600-2023073112/SouthWestCoast/370483_8.csv  
2   /Users/qyang/Downloads/data/CREST/CREST_tutorial/demo_project/Web_data/2023072600-2023073112/SouthWestCoast/370483_9.csv  
3  /Users/qyang/Downloads/data/CREST/CREST_tutorial/demo_project/Web_data/202307260

## 4. Create the web

### 4.1 Web for the forecasted streamflow.

In [52]:
import folium
from folium.plugins import MarkerCluster, MousePosition

# If needed, reproject your geodataframe to EPSG:4326 (lat/lon)
gdf = gdf.to_crs(epsg=4326)

# Initialize a new map
m = folium.Map(location=[merged['geometry'].y.mean(), merged['geometry'].x.mean()], zoom_start=5)

# Set color scheme for the points
colormap = folium.LinearColormap(['lightgray', 'green', 'blue', 'red'], vmin=merged['Peak_R'].min(), vmax=merged['Peak_R'].max())

# Create a cluster of markers, this helps in case of large number of points
marker_cluster = MarkerCluster().add_to(m)

# Display the latitude and longitude of the mouse position.
formatter = "function(num) {return L.Util.formatNum(num, 3);};"
mouse_position = MousePosition(
    position='topright',
    separator=' | ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Coordinates:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)
m.add_child(mouse_position)

# Iterate over each row in the dataframe
for idx, row in merged.iterrows():
    # Get lat and lon for each point
    lat = row['geometry'].y
    lon = row['geometry'].x

    # Get information for each point
    out_name = row['OutName']
    peak_r = row['Peak_R']
    date = row['Date']

    # Create the html for the popup
    html = f'<h3>{out_name}</h3><p>Peak_R: {peak_r}<br>Date: {date}</p>'

    # Create a popup with the html
    popup = folium.Popup(html, max_width=300)

    # Create a CircleMarker with color based on 'Peak_R' value
    marker = folium.CircleMarker(
        location=[lat, lon],
        radius=10,  # Adjusted the radius to enlarge the CircleMarker
        popup=popup,
        fill=True,
        color=colormap(peak_r),  # Here colormap is used to get color based on 'Peak_R' value
        fill_color=colormap(peak_r),
        fill_opacity=1.0
    )

    # Add the marker to the cluster
    marker.add_to(marker_cluster)

# Add the color map to the map
colormap.add_to(m)

# Add Tile Layers to the map
folium.TileLayer('openstreetmap').add_to(m)
folium.TileLayer('Stamen Terrain').add_to(m)
folium.TileLayer('Stamen Toner').add_to(m)
folium.LayerControl().add_to(m)

# Directly show the map in notebook
m


### 4.2 Overlap with observed flood locations.

Create random oberserved flood locations to the web.

In [53]:
import geopandas as gpd
import pandas as pd
import random
from shapely.geometry import Point
from folium.map import Marker

# Define the boundaries
lat_min, lat_max = 16.9, 19.27
lon_min, lon_max = 42.517, 45.75

# Generate 200 random lat and lon
random.seed(42)  # For reproducibility
lats = [random.uniform(lat_min, lat_max) for _ in range(200)]
lons = [random.uniform(lon_min, lon_max) for _ in range(200)]

# Create a list of shapely.Point out of these lats and lons
geometry = [Point(lon, lat) for lon, lat in zip(lons, lats)]

# Create a new DataFrame and convert it to a GeoDataFrame
df_flood = pd.DataFrame({'Obs_flood_location': [1]*200})
gdf_flood = gpd.GeoDataFrame(df_flood, geometry=geometry)

# Project the new GeoDataFrame to EPSG:4326
gdf_flood = gdf_flood.set_crs(epsg=4326)

# Iterate over each row in the new GeoDataFrame
for idx, row in gdf_flood.iterrows():
    # Get lat and lon for each point
    lat = row['geometry'].y
    lon = row['geometry'].x

    # Create a popup
    popup = folium.Popup('Obs Flood Locations', max_width=300)

    # Create a Marker and add it to the map
    marker = Marker(
        location=[lat, lon],
        popup=popup,
        icon=folium.Icon(color='blue', icon='cloud')
    )
    marker.add_to(m)

# Update the LayerControl and show the map
folium.LayerControl().add_to(m)
m
